Template notebook for one type of model.

In [87]:
import pandas as pd
from xgboost import XGBClassifier,XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from xgboost import XGBRegressor, XGBClassifier
from statistics import mean
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
# Other imports

In [88]:
# Load data
data= pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")
data_test = pd.read_csv("test_data_enriched.csv",index_col="Claim Identifier")

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/1043285265.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")


In [89]:
le = LabelEncoder()

In [90]:
#Label inconding our target variable 
data["Claim Injury Type"] = le.fit_transform(data["Claim Injury Type"])

In [91]:
def impute_missing_values(data, target_column, algorithm):
    # Separating the missing values from the non missing values
    available_data = data[data[target_column].notna()]
    missing_data = data[data[target_column].isna()]

    # Diagnóstico inicial
    print(f"\nImputando valores para coluna: {target_column}")
    print(f"available data: {len(available_data)}")
    print(f"missing data: {len(missing_data)}")

    # Verificar se há dados suficientes para imputação
    if len(available_data) == 0 or len(missing_data) == 0:
        print(f"nao deu input na coluna {target_column}")
        return data

    # Separating the target column from the rest
    X_available = available_data.drop(columns=[target_column])
    y_available = available_data[target_column]

    # Garantir consistência entre colunas
    X_available = X_available.select_dtypes(include=["number"])
    X_missing = missing_data.drop(columns=[target_column]).select_dtypes(include=["number"])
    common_columns = X_available.columns.intersection(X_missing.columns)
    X_available = X_available[common_columns]
    X_missing = X_missing[common_columns]

    # Verificar se ainda há colunas suficientes após alinhamento
    if X_available.shape[1] == 0:
        print(f"Sem colunas disponíveis para imputar na coluna {target_column}")
        return data

    # Training the model with the available data
    model = algorithm
    model.fit(X_available, y_available)

    # Prediting the missing values
    predicted_values = model.predict(X_missing)

    # Inputing the missing values with the predictions
    data.loc[data[target_column].isna(), target_column] = predicted_values

    return data

In [92]:
def impute_missing_values_with_fallback(data, target_column, algorithm):
    available_data = data[data[target_column].notna()]
    missing_data = data[data[target_column].isna()]

    if len(available_data) == 0 or len(missing_data) == 0:
        print(f"Sem dados suficientes para imputar a coluna '{target_column}', usando mediana.")
        data[target_column].fillna(data[target_column].median(), inplace=True)
        return data

    X_available = available_data.drop(columns=[target_column]).select_dtypes(include=["number"])
    y_available = available_data[target_column]
    X_missing = missing_data.drop(columns=[target_column]).select_dtypes(include=["number"])

    common_columns = X_available.columns.intersection(X_missing.columns)
    X_available = X_available[common_columns]
    X_missing = X_missing[common_columns]

    if X_available.shape[1] == 0 or X_missing.shape[1] == 0:
        print(f"Sem colunas suficientes para imputar '{target_column}', usando mediana.")
        data[target_column].fillna(data[target_column].median(), inplace=True)
        return data

    model = algorithm
    model.fit(X_available, y_available)
    predicted_values = model.predict(X_missing)
    data.loc[data[target_column].isna(), target_column] = predicted_values
    return data

In [93]:
def check_missing_values(data, step_name):
    print(f"\n{step_name}: Valores ausentes restantes:")
    print(data.isnull().sum()[data.isnull().sum() > 0])

In [94]:
def handle_outliers(data, column):
    if data[column].notnull().sum() > 0:  # Apenas processa colunas com dados válidos
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Substituir valores fora dos limites pelos próprios limites
        data[column] = np.where(data[column] < lower_bound, lower_bound, data[column])
        data[column] = np.where(data[column] > upper_bound, upper_bound, data[column])

In [95]:
def scale_numerical(column, X_train, X_val, scaler):
    # Certifique-se de que a coluna é numérica
    if not pd.api.types.is_numeric_dtype(X_train[column]):
        print(f"A coluna '{column}' não é numérica e será ignorada.")
        return

    # Escala os dados e substitui os valores na coluna
    try:
        X_train[column] = scaler.fit_transform(X_train[[column]].values.reshape(-1, 1))
        X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
    except ValueError as e:
        print(f"Erro ao escalonar a coluna '{column}': {e}")

In [96]:
def  claim_carrier_categories(X_train, X_val):
    # Define a function to categorize each carrier based on its claim count
    count = X_train['Carrier Name'].value_counts()
    def categorize_claims(count):
        if count >= 40000:
            return 2
        elif 4000 <= count < 40000:
            return 1
        else:
            return 0

    # Apply the categorization to create a mapping dictionary
    carrier_category_map = count.apply(categorize_claims)

    # Map the `Carrier Name` to the new `Carrier Claim Category`
    X_train['Carrier Claim Category'] = X_train['Carrier Name'].map(carrier_category_map)
    X_val['Carrier Claim Category'] = X_val['Carrier Name'].map(carrier_category_map)


In [97]:
# Categorical encoder function
def categorical_prop_encode(X_train, X_val, feature):
    proportion = X_train[feature].value_counts(normalize = True)  # Get the porportion of each category
    X_train[feature] = X_train[feature].map(proportion)  # Map the porportions in the column
    X_val[feature] = X_val[feature].map(proportion) # Do the same for the valid dataset
    X_val[feature] = X_val[feature].fillna(0)  # Handle categories in X_val not seen in X_train with 0


In [98]:
"""def Rfe(algorithm, num_inputing_algorithm= DecisionTreeRegressor() , cat_inputing_algorithm = DecisionTreeClassifier(), scaling_outlier = True):

    X_train, X_val,y_train, y_val = train_test_split(X,y,
                                                train_size = 0.8, 
                                                shuffle = True, 
                                                stratify = y)

    #Filling num missing values
    for column in num_features:
        impute_missing_values(X_train, column, num_inputing_algorithm)
        impute_missing_values(X_val, column, num_inputing_algorithm)

    #Filling cat missing values
    impute_missing_values(X_train, "Alternative Dispute Resolution", cat_inputing_algorithm)
    impute_missing_values(X_val, "Alternative Dispute Resolution", cat_inputing_algorithm)

    # Removing inconsistencies on the train
    inconsistent = X_train[(X_train['Age at Injury'] > 80) | (X_train["Age at Injury"] < 16)].index
    X_train = X_train.loc[~X_train.index.isin(inconsistent)]
    y_train = y_train.loc[~y_train.index.isin(inconsistent)]

    #Performing scaling and outlier treatment dependent on the boolean
    if scaling_outlier:
        for column in num_features:
            handle_outliers(X_train, column)
            impute_missing_values(X_train, column, num_imputing_algorithm)
                
        for column in num_features:
            scale_numerical(column,X_train, X_val, scaler)

    # Creating an ordinal variable
    claim_carrier_categories(X_train, X_val)

    impute_missing_values(X_val, "Carrier Claim Category", cat_inputing_algorithm)

    # Categorical Prop Encoding
    for cat_feature in cat_features:
        categorical_prop_encode(X_train, X_val, cat_feature)
    
    #Generating the variables where we will store our results
    nof_list = np.arange(1, len(X_train.columns) + 1)            
    high_score = 0
    opt_n_features = 0
    train_score_list = []
    val_score_list = []

    #Variable where we will store the optimum amount of features
    best_rfe = None

    model = algorithm

    for n in nof_list:
        rfe = RFE(estimator=model, n_features_to_select=n)
    
    # Fitting the model to rfe
        X_train_rfe = rfe.fit_transform(X_train, y_train)
        X_val_rfe = rfe.transform(X_val)
    
    # Training and predicting
        model.fit(X_train_rfe, y_train)
        pred_train = model.predict(X_train_rfe)
        pred_val = model.predict(X_val_rfe)
    
    # Evaluating using the macro f1_score
        train_score = f1_score(y_train, pred_train, average="macro")
        val_score = f1_score(y_val, pred_val, average="macro")
        train_score_list.append(train_score)
        val_score_list.append(val_score)
    
    # Checking if this is the best combination of features so far
        if val_score >= high_score:
            high_score = val_score
            opt_n_features = n
            best_rfe = rfe  # Armazenar o RFE com o melhor número de features

# Checking what amount of features and which features where the best for the model
    selected_features = X_train.columns[best_rfe.support_].tolist()

    print("Optimal number of features: %d" % opt_n_features)
    print("Score with %d features: %f" % (opt_n_features, high_score))
    print("Selected Features:\n", selected_features)
"""

'def Rfe(algorithm, num_inputing_algorithm= DecisionTreeRegressor() , cat_inputing_algorithm = DecisionTreeClassifier(), scaling_outlier = True):\n\n    X_train, X_val,y_train, y_val = train_test_split(X,y,\n                                                train_size = 0.8, \n                                                shuffle = True, \n                                                stratify = y)\n\n    #Filling num missing values\n    for column in num_features:\n        impute_missing_values(X_train, column, num_inputing_algorithm)\n        impute_missing_values(X_val, column, num_inputing_algorithm)\n\n    #Filling cat missing values\n    impute_missing_values(X_train, "Alternative Dispute Resolution", cat_inputing_algorithm)\n    impute_missing_values(X_val, "Alternative Dispute Resolution", cat_inputing_algorithm)\n\n    # Removing inconsistencies on the train\n    inconsistent = X_train[(X_train[\'Age at Injury\'] > 80) | (X_train["Age at Injury"] < 16)].index\n    X_train = 

In [99]:
def cv_scores(model, X, y, num_features, cat_features, num_imputing_algorithm= XGBRegressor() , cat_imputing_algorithm = XGBClassifier(), scaling_outlier = True, scaler = MinMaxScaler()):
    ''' Takes as argument the model used, the predictors and the target, the models used for imputing numerical and categorical 
      features, if any scaling and outlier removed should be performed, and what scaling method should be used.
     Then it returns the results obtained from the stratified cross validation for the given model'''
    
    skf = StratifiedKFold(n_splits=5)
    
    # Generating the lists to store our results
    precision_scores_train = []
    precision_scores_val = []   
    recall_scores_train = []  
    recall_scores_val = []
    f1_scores_train = []    
    f1_scores_val = []
    index = [f'Fold {i}' for i in range(1,6)]
    index.append("Average")
    
    for train_index, test_index in skf.split(X, y):
        # Dividing our data in validation and train
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]

 
        #Filling num missing values
        for column in num_features:
            impute_missing_values(X_train, column, num_imputing_algorithm)
            impute_missing_values(X_val, column, num_imputing_algorithm)

        #Filling cat missing values
        impute_missing_values(X_train, "Alternative Dispute Resolution", cat_imputing_algorithm)
        impute_missing_values(X_val, "Alternative Dispute Resolution", cat_imputing_algorithm)


        # Removing inconsistencies on the train
        inconsistent = X_train[(X_train['Age at Injury'] > 80) | (X_train["Age at Injury"] < 16)].index
        X_train = X_train.loc[~X_train.index.isin(inconsistent)]
        y_train = y_train.loc[~y_train.index.isin(inconsistent)]

        #Performing scaling and outlier treatment dependent on the boolean
        if scaling_outlier:
            for column in num_features:
                handle_outliers(X_train, column)
                impute_missing_values(X_train, column, num_imputing_algorithm)
                
            for column in num_features:
                scale_numerical(column,X_train, X_val, scaler)

            check_missing_values(X, "Apos scalling")

        # Creating an ordinal variable
        claim_carrier_categories(X_train, X_val)
        print(X_train["Carrier Claim Category"].value_counts())
        print(X_val["Carrier Claim Category"].value_counts())

        impute_missing_values(X_val, "Carrier Claim Category", cat_imputing_algorithm)


        # Categorical Prop Encoding
        for cat_feature in cat_features:
            categorical_prop_encode(X_train, X_val, cat_feature)
            
        
        # Training the classification model
        model.fit(X_train, y_train)

        
        # Making the predictions for the training and validation data
        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)
    
        
        # Calculating and storing the scores
        precision_scores_train.append(precision_score(y_train, pred_train, average='macro'))
        precision_scores_val.append(precision_score(y_val, pred_val, average='macro'))
        recall_scores_train.append(recall_score(y_train, pred_train, average='macro'))
        recall_scores_val.append(recall_score(y_val, pred_val, average='macro'))
        f1_scores_train.append(f1_score(y_train, pred_train, average='macro'))
        f1_scores_val.append(f1_score(y_val, pred_val, average='macro'))

    
    precision_scores_train.append(mean(precision_scores_train))
    precision_scores_val.append(mean(precision_scores_val))
    recall_scores_train.append(mean(recall_scores_train))
    recall_scores_val.append(mean(recall_scores_val))
    f1_scores_train.append(mean(f1_scores_train))
    f1_scores_val.append(mean(f1_scores_val))


    # Storing the results in a dataframe
    model_results = pd.DataFrame(data={
        'Train_precision': precision_scores_train,
        'Test_precision': precision_scores_val,
        'Train_recall': recall_scores_train,
        'Test_recall': recall_scores_val,
        'Train_f1_score': f1_scores_train,
        'Test_f1_score': f1_scores_val,
    }, index=index)
    
    return model_results



In [100]:
def test_prediction(model, X, y, num_features, cat_features, data_test, 
                    num_inputing_algorithm=XGBRegressor(), 
                    cat_inputing_algorithm=XGBClassifier(), 
                    scaler=MinMaxScaler()):

    # Etapa 1: Imputação inicial de valores ausentes
    for column in num_features:
        impute_missing_values_with_fallback(X, column, num_inputing_algorithm)
        impute_missing_values_with_fallback(data_test, column, num_inputing_algorithm)
    impute_missing_values_with_fallback(X, "Alternative Dispute Resolution", cat_inputing_algorithm)
    impute_missing_values_with_fallback(data_test, "Alternative Dispute Resolution", cat_inputing_algorithm)

    # Etapa 2: Remoção de inconsistências
    inconsistent = X[(X['Age at Injury'] > 80) | (X["Age at Injury"] < 16)].index
    X.drop(inconsistent, inplace=True)
    y.drop(inconsistent, inplace=True)

    # Etapa 3: Tratamento de outliers
    for column in num_features:
        handle_outliers(X, column)
        handle_outliers(data_test, column)

    # Etapa 4: Reimputação Após Outliers
    for column in num_features:
        impute_missing_values_with_fallback(X, column, num_inputing_algorithm)
        impute_missing_values_with_fallback(data_test, column, num_inputing_algorithm)

    # Etapa 5: Escalonamento
    for column in num_features:
        # Diagnóstico antes do escalonamento
        if X[column].isnull().sum() > 0:
            impute_missing_values_with_fallback(X, column, num_inputing_algorithm)
        scale_numerical(column, X, data_test, scaler)

    for cat_feature in cat_features:
        categorical_prop_encode(X, data_test, cat_feature)

    # Diagnóstico final
    check_missing_values(X, "Antes do Treinamento Final")

    model.fit(X, y)
    pred_test = model.predict(data_test)
    pred_test = le.inverse_transform(pred_test)

    submission_df = pd.DataFrame({
        "Claim Injury Type": pred_test
    }, index=data_test.index)
    
    return submission_df

Temos de rever as num features e cat features

In [101]:
num_features = ['Age at Injury', 'Average Weekly Wage', 'Birth Year', 'IME-4 Count', 'Number of Dependents',
                "Accident Year","Accident Month","Accident Day","Accident DayOfWeek","Assembly Date DSA",
                "C-2 Date DSA","C-3 Date DSA","First Hearing Date DSA" ,"Accident Year",
    "Accident Month",
    "Accident Day",
    "Accident DayOfWeek","Accident Date","C-3 Date","First Hearing Date"]

In [102]:
cat_features = [
    "Alternative Dispute Resolution",
    "Carrier Name",
    "Carrier Type",
    "County of Injury",
    "District Name",
    "Gender",
    "Industry Code",
    "Medical Fee Region",
    "WCIO Cause of Injury Code",
    "WCIO Nature of Injury Code",
    "WCIO Part Of Body Code",
    "Age at Injury Category",
    "Carrier Claim Category",
    "Body Section"
]

temos de dropar as colunas e por no pre-processing

In [103]:
# Dropping redundant variables that carry almost the same information (are extremely correlated (|0.8|))
# We believe it was better to keep Age at Injury than birth year since it should be more related to the injury claim type (it will be tested later)
# The same logic was applied to dropping the other two dates and two DSA variables since we believe Accident date to be more important
# Para `data`
data = data.loc[:, ~data.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]

# Para `data_test`
data_test = data_test.loc[:, ~data_test.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]


for col in ['Birth Year',"Assembly Date DSA", "First Hearing Date DSA"]:
    num_features.remove(col)

In [104]:
#Since the codes always seem to provide the same or more information than the descriptions (have more categories),
#and the codes are consistent (always only having 1 description for code, while descriptions may have multiple codes)
#we will drop the description columns.
data.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)
data_test.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)


In [105]:
# Removing Zip Code for reason meantion in pre-processement
data.drop(['Zip Code'], axis=1, inplace = True)
data_test.drop(['Zip Code'], axis=1 , inplace = True)

In [106]:
X = data.drop(["Claim Injury Type"], axis = 1)

In [107]:
y = data["Claim Injury Type"]

In [ ]:
# ------------------------

In [ ]:
# Select Categorical features
selected_cat_features = ['',''] # based on RFE

# Select numerical features
selected_num_features = ['Age at Injury', 'Average Weekly Wage','IME-4 Count','Number of Dependents',
                         'Accident Year','Accident Month', 'Accident Day', 'Accident DayOfWeek',
                         'C-2 Date DSA', 'C-3 Date DSA'] # based on RFE

In [108]:
xgb_results = cv_scores(XGBClassifier(), X, y,num_features,cat_features,scaling_outlier = True)
# look at results


Imputando valores para coluna: Age at Injury
available data: 455208
missing data: 4012

Imputando valores para coluna: Age at Injury
available data: 113354
missing data: 1452

Imputando valores para coluna: Average Weekly Wage
available data: 168515
missing data: 290705

Imputando valores para coluna: Average Weekly Wage
available data: 41410
missing data: 73396

Imputando valores para coluna: IME-4 Count
available data: 107670
missing data: 351550

Imputando valores para coluna: IME-4 Count
available data: 25133
missing data: 89673

Imputando valores para coluna: Number of Dependents
available data: 459220
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Number of Dependents
available data: 114806
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Accident Year
available data: 456463
missing data: 2757

Imputando valores para coluna: Accident Year
available data: 113874
missing data: 932

Imputando 

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value 


Imputando valores para coluna: Age at Injury
available data: 455036
missing data: 4185

Imputando valores para coluna: Age at Injury
available data: 113526
missing data: 1279

Imputando valores para coluna: Average Weekly Wage
available data: 168185
missing data: 291036

Imputando valores para coluna: Average Weekly Wage
available data: 41740
missing data: 73065

Imputando valores para coluna: IME-4 Count
available data: 106606
missing data: 352615

Imputando valores para coluna: IME-4 Count
available data: 26197
missing data: 88608

Imputando valores para coluna: Number of Dependents
available data: 459221
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Number of Dependents
available data: 114805
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Accident Year
available data: 456408
missing data: 2813

Imputando valores para coluna: Accident Year
available data: 113929
missing data: 876

Imputando 

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value 


Imputando valores para coluna: Age at Injury
available data: 454785
missing data: 4436

Imputando valores para coluna: Age at Injury
available data: 113777
missing data: 1028

Imputando valores para coluna: Average Weekly Wage
available data: 167921
missing data: 291300

Imputando valores para coluna: Average Weekly Wage
available data: 42004
missing data: 72801

Imputando valores para coluna: IME-4 Count
available data: 105844
missing data: 353377

Imputando valores para coluna: IME-4 Count
available data: 26959
missing data: 87846

Imputando valores para coluna: Number of Dependents
available data: 459221
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Number of Dependents
available data: 114805
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Accident Year
available data: 456192
missing data: 3029

Imputando valores para coluna: Accident Year
available data: 114145
missing data: 660

Imputando 

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value 


Imputando valores para coluna: Age at Injury
available data: 454689
missing data: 4532

Imputando valores para coluna: Age at Injury
available data: 113873
missing data: 932

Imputando valores para coluna: Average Weekly Wage
available data: 167705
missing data: 291516

Imputando valores para coluna: Average Weekly Wage
available data: 42220
missing data: 72585

Imputando valores para coluna: IME-4 Count
available data: 105589
missing data: 353632

Imputando valores para coluna: IME-4 Count
available data: 27214
missing data: 87591

Imputando valores para coluna: Number of Dependents
available data: 459221
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Number of Dependents
available data: 114805
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Accident Year
available data: 456174
missing data: 3047

Imputando valores para coluna: Accident Year
available data: 114163
missing data: 642

Imputando v

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value 


Imputando valores para coluna: Age at Injury
available data: 454530
missing data: 4691

Imputando valores para coluna: Age at Injury
available data: 114032
missing data: 773

Imputando valores para coluna: Average Weekly Wage
available data: 167374
missing data: 291847

Imputando valores para coluna: Average Weekly Wage
available data: 42551
missing data: 72254

Imputando valores para coluna: IME-4 Count
available data: 105503
missing data: 353718

Imputando valores para coluna: IME-4 Count
available data: 27300
missing data: 87505

Imputando valores para coluna: Number of Dependents
available data: 459221
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Number of Dependents
available data: 114805
missing data: 0
nao deu input na coluna Number of Dependents

Imputando valores para coluna: Accident Year
available data: 456111
missing data: 3110

Imputando valores para coluna: Accident Year
available data: 114226
missing data: 579

Imputando v

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[column] = scaler.transform(X_val[[column]].values.reshape(-1, 1))
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_1692/530874828.py:10: SettingWithCopyWarning: 
A value 

In [109]:
xgb_results

,Train_precision,Test_precision,Train_recall,Test_recall,Train_f1_score,Test_f1_score
Fold 1,0.790565,0.355897,0.593245,0.404946,0.630120,0.304148
Fold 2,0.802209,0.280416,0.619805,0.259754,0.657855,0.259233
Fold 3,0.797484,0.239749,0.622445,0.211982,0.660535,0.216118
Fold 4,0.802551,0.254437,0.602355,0.218395,0.636726,0.226064
Fold 5,0.806699,0.333180,0.575631,0.206590,0.609947,0.154767
Average,0.799902,0.292736,0.602696,0.260333,0.639037,0.232066


In [ ]:
# Test prediction
    # Choose best model from KFold above
    # Train model on whole train and predict test data
submission = test_prediction(XGBClassifier(),X,y,num_features,cat_features,data_test)
submission

In [ ]:
submission.nunique()